In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import csv
from joblib import Parallel, delayed
import multiprocessing

In [8]:
def get_pages(main):
    try:
        soup = connect(main)
        n_pages = [_.get_text(strip=True) for _ in soup.find('ul', {'class': 'pagination pagination__number'}).find_all('li')]
        #max = soup.find_all("span", class_="pagination__number")
        last_page = int(n_pages[-1])
        pages = [main]
        
        for n in range(2,last_page+1):    
            page_num = "/?pag={}".format(n)
            pages.append(main + page_num)
    except:
        pages = [main]
        
    return pages

def connect(web_addr):
    resp = requests.get(web_addr)
    return BeautifulSoup(resp.content, "html.parser")
    

def get_areas(website):
    data = connect(website)
    areas = []
    for ultag in data.find_all('ul', {'class': 'breadcrumb-list breadcrumb-list_list breadcrumb-list__related'}):
        for litag in ultag.find_all('li'):
            for i in range(len(litag.text.split(','))):
                areas.append(litag.text.split(',')[i])
    areas = [x.strip() for x in areas]
    urls = []
    
    for area in areas:
        url = website + '/' + area.replace(' ','-').lower()
        urls.append(url)
    
    return areas, urls

def get_apartment_links(website):
    data = connect(website)
    links = []
    for link in data.find_all('ul', {'class': 'annunci-list'}):
        for litag in link.find_all('li'):
            try:
                #return litag.a.get('href')
                links.append(litag.a.get('href'))
            except:
                continue
    return links

def scrape_link(website):
    data = connect(website)
    nomi = []
    valori = []
    temp = []
    for link in (data.find_all('dd', {'class': 'im-features__value'})[1], data.find_all('dd', {'class': 'im-features__value'})[9], data.find_all('dd', {'class': 'im-features__value'})[12]):
        temp.append(link)
        try:
            temp = str(temp).replace('</dd>', '')
            temp = str(temp).replace('<dt', '</dd><dt')
            temp = BeautifulSoup(temp, "html.parser")
            for elem in temp.find_all('dd'):
                valori.append(elem.string.strip())
        except:
            #print('errore')
            pass
        temp = []

    valori[-1] = (data.find_all('dd', {'class': 'im-features__value'})[0].find_all('span')[0].string)

    for link in data.find_all('dt'):
        if link.string == None:
            pass
        else:
            nomi.append(link.string.strip())

    while len(valori)<len(nomi):
        valori.append(0)

    while len(nomi)<len(valori):
        nomi.append('Prestazione energetica del fabbricato')

    count = 0
    nomi.append('Arredato S/N')
    nomi.remove('altre caratteristiche')
    for elem in data.find_all('dd', {'class': 'im-features__value'})[8].find_all('span'):
        if elem.string.strip() == "Arredato" or elem.string == "Non arredato" or elem.string == "Parzialmente arredato":
            valori.append(elem.string.strip())
            count = 1            
        else:
            continue
    if count == 0:
        valori.append('0')        

    valori = remove_duplicates(valori)
    nomi = remove_duplicates(nomi)
    
    return nomi, valori

def remove_duplicates(x):
    return list(dict.fromkeys(x))


In [9]:
## Link for city main website
## get areas inside the city (districts)

website = "https://www.immobiliare.it/affitto-case/torino"
areas, districts = get_areas(website)
print("Those are district's links \n")
print(districts)

Those are district's links 

['https://www.immobiliare.it/affitto-case/torino/aurora', 'https://www.immobiliare.it/affitto-case/torino/barriera-di-milano', 'https://www.immobiliare.it/affitto-case/torino/rebaudengo', 'https://www.immobiliare.it/affitto-case/torino/barriera-di-lanzo', 'https://www.immobiliare.it/affitto-case/torino/falchera', 'https://www.immobiliare.it/affitto-case/torino/barca', 'https://www.immobiliare.it/affitto-case/torino/bertolla', 'https://www.immobiliare.it/affitto-case/torino/borgo-san-paolo', 'https://www.immobiliare.it/affitto-case/torino/cenisia', 'https://www.immobiliare.it/affitto-case/torino/borgo-vittoria', 'https://www.immobiliare.it/affitto-case/torino/parco-dora', 'https://www.immobiliare.it/affitto-case/torino/campidoglio', 'https://www.immobiliare.it/affitto-case/torino/san-donato', 'https://www.immobiliare.it/affitto-case/torino/cit-turin', 'https://www.immobiliare.it/affitto-case/torino/cavoretto', 'https://www.immobiliare.it/affitto-case/torino/

## Scrape cycle initialization

In [11]:
indirizzo = []
location = []

for url in tqdm(districts):
    pages = get_pages(url)
    for page in pages:
        add = get_apartment_links(page)
        indirizzo.append(add)
        for num in range(0,len(add)):
            location.append(url.rsplit('/', 1)[-1])
        
announces_links = [item for valore in indirizzo for item in valore]

C:\Users\Francesco Manghi\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [13]:
## Check that what you scraped has a meaning...and save it

print("The numerosity of announces:\n")
print(len(announces_links))
with open('announces_list.csv', 'w') as myfile:
    wr = csv.writer(myfile)
    wr.writerow(announces_links)

The numerosity of announces:

3481


## Proper dataset creation by scraping every announce

In [14]:
## DATAFRAME ORIZZONTALE__USARE

#variabili = ['Riferimento e Data annuncio', 'Contratto', 'Tipologia', 'Superficie', 'Locali', 'Piano', 'Disponibilità', 'Tipo proprietà', 'Prezzo', 'Spese condominio', 'Spese riscaldamento', 'Informazioni catastali', 'Anno di costruzione', 'Stato', 'Riscaldamento', 'Climatizzatore', 'Classe energetica', 'Prestazione energetica del fabbricato', 'Arredato S/N']
#df_scrape = pd.DataFrame(columns=variabili)
df_scrape = pd.DataFrame()
#df_scrape.set_index(0, inplace=True, drop=True)
#print(df_scrape)
to_be_dropped = []
counter = 0
for link in tqdm(list(announces_links[0:10])):
    print(link)
    counter=counter+1
    try:
        nomi, valori = scrape_link(link)
        df_temporaneo = pd.DataFrame(columns=nomi)
        #df_temporaneo['Nomi'] = nomi
        #df_temporaneo.set_index('Nomi',inplace=True,drop=True)
        #print(len(nomi),len(valori))
        #while len(valori)<len(nomi):
        #    valori.append(0)
        #print(len(valori[0:len(nomi)]))
        df_temporaneo.loc[len(df_temporaneo), :] = valori[0:len(nomi)]
        #print(df_temporaneo)
        df_scrape = df_scrape.append(df_temporaneo, sort=False)
    except Exception as e:
        print(e)
        to_be_dropped.append(counter)
        print(to_be_dropped)
        continue

C:\Users\Francesco Manghi\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


https://www.immobiliare.it/annunci/80839413/
cannot copy sequence with size 11 to array axis with dimension 17
[1]
https://www.immobiliare.it/annunci/80835421/
cannot copy sequence with size 12 to array axis with dimension 14
[1, 2]
https://www.immobiliare.it/annunci/80828587/
cannot copy sequence with size 13 to array axis with dimension 17
[1, 2, 3]
https://www.immobiliare.it/annunci/80828403/
cannot copy sequence with size 13 to array axis with dimension 16
[1, 2, 3, 4]
https://www.immobiliare.it/annunci/80829815/
cannot copy sequence with size 13 to array axis with dimension 16
[1, 2, 3, 4, 5]
https://www.immobiliare.it/annunci/80826259/
cannot copy sequence with size 11 to array axis with dimension 14
[1, 2, 3, 4, 5, 6]
https://www.immobiliare.it/annunci/80792163/
cannot copy sequence with size 13 to array axis with dimension 17
[1, 2, 3, 4, 5, 6, 7]
https://www.immobiliare.it/annunci/80787725/
cannot copy sequence with size 11 to array axis with dimension 17
[1, 2, 3, 4, 5, 6, 7,

In [7]:
#for item in to_be_dropped:
pd.DataFrame(location).to_csv('location.csv', sep=';')
pd.DataFrame(to_be_dropped).to_csv('to_be_dropped.csv', sep=';')

In [8]:
to_be_dropped.sort(reverse=True)

In [9]:
to_be_dropped

[2816,
 2775,
 2742,
 2728,
 2714,
 2702,
 2660,
 2615,
 2605,
 2228,
 2182,
 2106,
 2100,
 1953,
 1931,
 1839,
 1830,
 1808,
 1771,
 1715,
 1677,
 1570,
 1532,
 1515,
 1487,
 1433,
 1336,
 1280,
 1129,
 1104,
 1081,
 1053,
 1021,
 968,
 942,
 841,
 794,
 634,
 610,
 434,
 390,
 311,
 258,
 85]

In [10]:
for index in to_be_dropped:
    del location[index-1]

In [11]:
for index in to_be_dropped:
    del announces_links[index-1]

In [12]:
len(location)

2405

In [21]:
print(df_scrape.shape)
df_scrape['zona'] = location
df_scrape.to_csv('dataset.csv', sep=";")

(2386, 16)


In [29]:
df_scrape = pd.read_csv('dataset.csv', sep=';')

In [30]:
df_scrape.columns

Index(['Unnamed: 0', 'riferimento e Data annuncio', 'tipologia', 'superficie',
       'locali', 'piano', 'disponibilità', 'Tipo proprietà', 'prezzo',
       'spese condominio', 'informazioni catastali', 'anno di costruzione',
       'stato', 'riscaldamento', 'Climatizzazione', 'Classe energetica',
       'Arredato S/N', 'zona'],
      dtype='object')

In [31]:
df_scrape = df_scrape[['tipologia', 'zona', 'tipologia', 'superficie', 'locali', 'piano', 'Tipo proprietà', 'prezzo', 'spese condominio','anno di costruzione', 'stato', 'riscaldamento', 'Climatizzazione', 'Classe energetica', 'Arredato S/N']]

In [34]:
def cleanup(df):
    price = []
    rooms = []
    surface = []
    bathrooms = []
    floor = []
    contract = []
    tipo = []
    condominio = []
    heating = []
    built_in = []
    state = []
    riscaldamento = []
    cooling = []
    energy_class = []
    tipologia = []
    pr_type = []
    arredato = []
    
    for tipo in df['tipologia']:
        try:
            tipologia.append(tipo)
        except:
            tipologia.append(None)
    
    for superficie in df['superficie']:
        try:
            if "m" in superficie:
                s = superficie.replace(" m²", "")
                surface.append(s)
        except:
            surface.append(None)
    
    for locali in df['locali']:
        try:
            rooms.append(locali[0:1])
        except:
            rooms.append(None)
    
    for prezzo in df['prezzo']:
        try:
            price.append(prezzo.replace("Affitto ", "").replace("€ ", "").replace(" al mese", "").replace(".",""))
        except:
            price.append(None)
            
    for contratto in df['tipologia']:
        try:
            contract.append(contratto.replace("\n ",""))
        except:
            contract.append(None)
    
    for piano in df['piano']:
        try:
            floor.append(piano.split(' ')[0])
        except:
            floor.append(None)
    
    for tipologia in df['Tipo proprietà']:
        try:
            pr_type.append(tipologia.split(',')[0])
        except:
            pr_type.append(None)
            
    for condo in df['spese condominio']:
        try:
            if "mese" in condo:
                condominio.append(condo.replace("€ ","").replace("/mese",""))
            else:
                condominio.append(None)
        except:
            condominio.append(None)
        
 #   for ii in df['Spese riscaldamento']:
 #       try:
 #           if "anno" in ii:
 #               mese = int(int(ii.replace("€ ","").replace("/anno","").replace(".",""))/12)
 #               heating.append(mese)
 #           else:
 #               heating.append(None)
 #       except:
 #           heating.append(None)
    
    for anno_costruzione in df['anno di costruzione']:
        try:
            built_in.append(anno_costruzione)
        except:
            built_in.append(None)
    
    for stato in df['stato']:
        try:
            stat = stato.replace(" ","").lower()
            state.append(stat)
        except:
            state.append(None)
    
    for tipo_riscaldamento in df['riscaldamento']:
        try:
            riscaldamento.append(tipo_riscaldamento.lower().split(',')[0])
        except:
            riscaldamento.append(None)
    
    for clima in df['Climatizzazione']:
        try:
            cooling.append(clima.lower().split(',')[0])
        except:
            cooling.append('None')
    
    for classe in df['Classe energetica']:
        try:
            energy_class.append(classe.replace("\n ",""))
        except:
            energy_class.append(None)
            
    for SN in df['Arredato S/N']:
        try:
            arredato.append(SN)
        except:
            arredato.append(None)
            
    
    final_df = pd.DataFrame(columns=['Contratto', 'Zona', 'Tipologia', 'Superficie', 'Locali', 'Piano', 'Tipo proprietà', 'Prezzo', 'Spese condominio', 'Spese riscaldamento','Anno di costruzione', 'Stato', 'Riscaldamento', 'Climatizzatore', 'Classe energetica', 'Arredato S/N'])
    final_df['Contratto'] = contract
    final_df['Tipologia'] = tipologia
    final_df['Superficie'] = surface
    final_df['Locali'] = rooms
    final_df['Piano'] = floor
    final_df['Tipo proprietà'] = pr_type
    final_df['Prezzo'] = price
    final_df['Spese condominio'] = condominio
    final_df['Spese riscaldamento'] = heating
    final_df['Anno di costruzione'] = built_in
    final_df['Stato'] = state
    final_df['Riscaldamento'] = riscaldamento
    final_df['Climatizzatore'] = cooling
    final_df['Classe energetica'] = energy_class
    final_df['Zona'] = df['Zona'].values
    final_df['Arredato S/N'] = arredato
    final_df['Link annuncio'] = announces_links
    
    return final_df

In [35]:
final = cleanup(df_scrape)
final.to_csv('dataset_da_allenamento.csv', sep=";")

ValueError: Length of values does not match length of index

In [36]:
surface

NameError: name 'surface' is not defined

## --------------------------

In [50]:
nomi, valori = scrape_link('https://www.immobiliare.it/annunci/79608977/')

IndexError: list index out of range

In [40]:
data = connect('https://www.immobiliare.it/annunci/80787725/')
nomi = []
valori = []
temp = []

for link in (data.find_all('dd', {'class': 'im-features__value'})[1], data.find_all('dd', {'class': 'im-features__value'})[9], data.find_all('dd', {'class': 'im-features__value'})[12]):
    temp.append(link)
    try:
        temp = str(temp).replace('</dd>', '')
        temp = str(temp).replace('<dt', '</dd><dt')
        temp = BeautifulSoup(temp, "html.parser")
        for elem in temp.find_all('dd'):
            valori.append(elem.string.strip())
    except:
        #print('errore')
        pass
    temp = []

valori[-1] = (data.find_all('dd', {'class': 'im-features__value'})[16].find_all('span')[0].string)

for link in data.find_all('dt'):
    if link.string == None:
        pass
    else:
        nomi.append(link.string.strip())

while len(valori)<len(nomi):
    valori.append(0)

while len(nomi)<len(valori):
    nomi.append('Prestazione energetica del fabbricato')

In [39]:
valori

['Affitto',
 'Appartamento',
 '60 m²',
 '2 (1 camera da letto, 1 altro), 1 bagno, cucina abitabile',
 '4° stabile',
 '4 piani',
 'Libero',
 'Intera proprietà, classe immobile media',
 'Classe A/3            ]']

In [41]:
nomi

['riferimento e Data annuncio',
 'contratto',
 'tipologia',
 'superficie',
 'locali',
 'piano',
 'totale piani edificio',
 'disponibilità',
 'Tipo proprietà',
 'altre caratteristiche',
 'prezzo',
 'spese condominio',
 'informazioni catastali',
 'anno di costruzione',
 'stato',
 'riscaldamento',
 'Classe energetica']

In [60]:
data = connect('https://www.immobiliare.it/annunci/80787725/')
html = (data.find_all('dl', {'class': 'im-features__list'}))

In [62]:
html[0]

<dl class="im-features__list">
<dt class="im-features__title">riferimento e Data annuncio</dt>
<dd class="im-features__value">
                    120489 - 08/06/2020                    <dt class="im-features__title">contratto</dt>
<dd class="im-features__value">
                    Affitto                    <dt class="im-features__title">tipologia</dt>
<dd class="im-features__value">
                    Appartamento                    <dt class="im-features__title">superficie</dt>
<dd class="im-features__value">
                    60 m²                    <dt class="im-features__title">locali</dt>
<dd class="im-features__value">
                    2 (1 camera da letto, 1 altro), 1 bagno, cucina abitabile                    <dt class="im-features__title">piano</dt>
<dd class="im-features__value">
                    4° stabile                    <dt class="im-features__title">totale piani edificio</dt>
<dd class="im-features__value">
                    4 piani                    <d

In [37]:
for item in flat_list:
    print(item)
    print('--------------------')



--------------------
<dt class="im-features__title">riferimento e Data annuncio</dt>
--------------------


--------------------
<dd class="im-features__value">
                    120489 - 08/06/2020                    <dt class="im-features__title">contratto</dt>
<dd class="im-features__value">
                    Affitto                    <dt class="im-features__title">tipologia</dt>
<dd class="im-features__value">
                    Appartamento                    <dt class="im-features__title">superficie</dt>
<dd class="im-features__value">
                    60 m²                    <dt class="im-features__title">locali</dt>
<dd class="im-features__value">
                    2 (1 camera da letto, 1 altro), 1 bagno, cucina abitabile                    <dt class="im-features__title">piano</dt>
<dd class="im-features__value">
                    4° stabile                    <dt class="im-features__title">totale piani edificio</dt>
<dd class="im-features__value">
             

In [50]:
data.find_all('dd', {'class': 'im-features__value'})[16].find_all('span')

[<span class="im-features__energy" data-class="G">G</span>]

In [59]:
for link in data.find_all('dd', {'class': 'im-features__value'}):
    print(len(link))
    print(link)

4
<dd class="im-features__value">
                    120489 - 08/06/2020                    <dt class="im-features__title">contratto</dt>
<dd class="im-features__value">
                    Affitto                    <dt class="im-features__title">tipologia</dt>
<dd class="im-features__value">
                    Appartamento                    <dt class="im-features__title">superficie</dt>
<dd class="im-features__value">
                    60 m²                    <dt class="im-features__title">locali</dt>
<dd class="im-features__value">
                    2 (1 camera da letto, 1 altro), 1 bagno, cucina abitabile                    <dt class="im-features__title">piano</dt>
<dd class="im-features__value">
                    4° stabile                    <dt class="im-features__title">totale piani edificio</dt>
<dd class="im-features__value">
                    4 piani                    <dt class="im-features__title">disponibilità</dt>
<dd class="im-features__value">
             